In [1]:
import time
import sys
import math
import numpy as np
from pyomo.environ import *
from pyomo.opt import SolverStatus, TerminationCondition
import pyomo.opt
import cvxopt as cvx #used by picos
import picos as pic #can solve SDP
import networkx as nx #manipulate graphs

In [5]:
GWitn = 10000 #max iteration of randomized rounding of GW
MyEpsilon = 1e-6 # tolerance
cplexTimeLimit= 100 #maximum seconds of user CPU for CPLEX

#Quadratic programming : pyomo
def create_maxcut_miqp(G) : 
    maxcut = ConcreteModel()
    # vars = 0 if node in S, 1 if not
    maxcut.y = Var(G.nodes(), within = Binary)
    
    #objective fn
    def max_cut(model):
        return sum(G[i][j]['weight'] * (model.y[i]+model.y[j] - \
                   2*model.y[i]*model.y[j]) for (i,j) in G.edges())
    maxcut.maxcutobj = Objective(rule = max_cut, sense = maximize)
    return maxcut

#SDP relaxation : picos
def create_maxcut_sdp(Laplacian):
    n = Laplacian.shape[0]
    maxcut = pic.Problem()
    X = maxcut.add_variable('X', (n,n), 'symmetric')
    maxcut.add_constraint(pic.tools.diag_vect(X) == 1)
    maxcut.add_constraint(X >> 0)
    L = pic.new_param('L', Laplacian)
    # max trace(LX)
    maxcut.set_objective('max', L|X)
    return maxcut

#Matrix factor V of X*    
def rank_factor(X, rk):
    n = X.shape[0]
    evl, evc = np.linalg.eigh(X)
    # set neg eigv to 0
    evl[evl<0] = 0
    if rk < n:
        V = np.transpose(evc[:,-rk:])
        for k in range(rk):
            V[k] *= math.sqrt(evl[-rk-1])
    else : 
        V = np.transpose(evc[::-1])
        for k in range(n):
            V[k] *= math.sqrt(evl[::-1][k])
    return V

# Gets closest sdp from X (force eigen > 0)    
def closest_sdp(X) : 
    evl, evc = np.linalg.eigh(X)
    evl[evl < 0] = 0
    return evc.T.dot(np.diag(evl).dot(evc))
    
def gw_randomized_rounding(V, objX, L, iterations):
    n = V.shape[0]
    count = 0
    obj = 0
    print("maxcut(GW/rnd) : impro_ithn objfun")
    while (count<iterations):
        # normalized vector on unit sphere
        r = np.random.normal(0,1,n)
        r /= np.linalg.norm(r) 
        
        # sign of V.r
        x = np.sign(np.dot(V,r))
        x[x>=0] = 1 # avoid 0
        
        # obj fn
        o = np.dot(x.T, np.dot(L,x))
        
        if o > obj : 
            x_cut = x
            obj = o
            print(str(count)+"\t"+str(obj))
        count +=1
    return (x_cut, obj)

In [8]:
G = nx.read_weighted_edgelist('data_chimera.nonweighted/3_0.edgelist', nodetype = int)
G = nx.Graph(G)

In [9]:
# MIQP formulation with Pyomo
miqp = create_maxcut_miqp(G)
solver = pyomo.opt.SolverFactory('cplexamp', solver_io = 'nl')
solver.options['timelimit'] = cplexTimeLimit
solver.options['mipdisplay'] = 2
t0 = time.time()
results= solver.solve(miqp, keepfiles = False, tee = True)
t1 = time.time()
miqpcpu = t1 - t0
miqp.solutions.load_from(results)
ymiqp = {i:miqp.y[i].value for i in G.nodes()}
miqpcut = [i for i, y in ymiqp.iteritems() if y == 1]
miqpobj = miqp.maxcutobj()

# SDP relaxation
L = np.array(1/4.*nx.laplacian_matrix(G).todense())
sdp = create_maxcut_sdp(L)
t0 = time.time()
sdp.solve(verbose = 0)
t1 = time.time()
sdpcpu = t1 - t0
sdpobj = sdp.obj_value()
Xsdp = closest_sdp(np.array(sdp.get_valued_variable('X')))
sdprank = np.linalg.matrix_rank(Xsdp)
sdpfull = Xsdp.shape[0]

#t0 = time.time() # because includes SDP time
N = len(G.nodes())
V = rank_factor(Xsdp, N)
(gwcut, gwobj) = gw_randomized_rounding(V, sdp.obj_value(), L, GWitn)
t1 = time.time()
gwcpu = t1 - t0

timelimit=100
mipdisplay=2
1 of 1 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective 0.0000.
MIP Presolve added 384 rows and 192 columns.
Reduced MIP has 384 rows, 264 columns, and 768 nonzeros.
Reduced MIP has 264 binaries, 0 generals, 0 SOSs, and 0 indicators.
Probing time = 0.00 sec. (0.06 ticks)
MIP Presolve eliminated 192 rows and 0 columns.
Reduced MIP has 192 rows, 264 columns, and 576 nonzeros.
Reduced MIP has 264 binaries, 0 generals, 0 SOSs, and 0 indicators.
Probing time = 0.00 sec. (0.04 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (0.49 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                            0.0000      384.0000              --- 
*     0     0      integral     0      

In [23]:
print(Xsdp)

import numpy as np
print(np.where(gwcut == 1)[0])

print(miqpcut)

[[ 0.49752328  0.25747885 -0.32999714 ...,  0.84941006  0.68702943
   0.7053533 ]
 [ 0.25747885  0.13325077 -0.17078052 ...,  0.43958772  0.3555523
   0.36503529]
 [-0.32999714 -0.17078052  0.21888044 ..., -0.56339653 -0.45569274
  -0.46784659]
 ..., 
 [ 0.84941006  0.43958772 -0.56339653 ...,  1.45017828  1.17294956
   1.20423348]
 [ 0.68702943  0.3555523  -0.45569274 ...,  1.17294956  0.94871831
   0.97402171]
 [ 0.7053533   0.36503529 -0.46784659 ...,  1.20423348  0.97402171  1.        ]]
[ 1  5  6  7 11 13 15 18 20 22 23 25 26 30 31 32 33 34 35 44 45 46 47 49 51
 52 54 55 56 57 58 68 69 70]
[4, 5, 6, 7, 8, 9, 10, 11, 20, 21, 22, 23, 24, 25, 26, 27, 36, 37, 38, 39, 40, 41, 42, 43, 52, 53, 54, 55, 56, 57, 58, 59, 68, 69, 70, 71]


In [35]:
import random
def gw_randomized_rounding_improved(V, objX, L, iterations):
    n = V.shape[0]
    count = 0
    obj = 0
    print("maxcut(GW/rnd) : impro_ithn objfun")
    while (count<iterations):
        # normalized vector on unit sphere
        r = np.random.normal(0,1,n)
        r /= np.linalg.norm(r) 

        # sign of V.r
        x = np.sign(np.dot(V,r))
        x[x>=0] = 1 # avoid 0
        for i in np.arange(0,len(x),4):
            rn = random.randint(0,3)
            x[i]=x[i+rn]
            x[i+1]=x[i+rn]
            x[i+2]=x[i+rn]
            x[i+3]=x[i+rn]
        
        # obj fn
        o = np.dot(x.T, np.dot(L,x))
        
        if o > obj : 
            x_cut = x
            obj = o
            print(str(count)+"\t"+str(obj))
        count +=1
    return (x_cut, obj)

In [38]:
N = len(G.nodes())
V = rank_factor(Xsdp, N)
(gwcut2, gwobj2) = gw_randomized_rounding_improved(V, sdp.obj_value(), L, GWitn)

print(miqpopt)

maxcut(GW/rnd) : impro_ithn objfun
0	68.0
1	104.0
6	108.0
9	120.0
11	124.0
27	156.0
311	176.0
5257	180.0
192.0
